# Extended functions on annotations

This jupyter notebook is written to allow extra visualisation capabilities on the created annotations, aside from format conversions and counting as provided in *annotations.ipynb*. 

Functions here will be expanded as the project progresses.

### Visualisation
* V1 - visualise detected instances

Utilities for creating dataFrames from annotation files

In [1]:
# import some common libraries
import pandas as pd
import numpy as np
import os, json, cv2
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from pycocotools import mask as cocomask

# utilities

def findCategory(data):
    # find categories
    cats = data["categories"]
    category = pd.DataFrame(cats)
    category = category.drop(['supercategory'], axis=1)
    category = category.rename(columns={'id': 'category_id'})
    return category

def findImages(data):
    img = data["images"]
    images = pd.DataFrame(img)
    
    # unwanted columns exist if exported from CVAT. Not if generated by my code
    if set(['license','flickr_url','coco_url','date_captured']).issubset(images.columns):
        images = images.drop(columns=['license','flickr_url','coco_url','date_captured'])
    
    return images

def findAnnotations(data):
    anno = data["annotations"]
    df = pd.DataFrame(anno)
    return df

def cleanForJson(category=None, df=None):
    # clean category for json dump
    if category is not None:
        category = category.rename(columns={'category_id': 'id'})
        category['supercategory'] = ""

    # add columns in df for json dump
    if df is not None:
        df['iscrowd'] = 0
        df['attributes'] = [{'occluded':False}] * len(df['id'])
        cols = ['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd', 'attributes']
        df = df[cols + [c for c in df.columns if c not in cols]]
    
    return category, df

# convert all np.integer, np.floating and np.ndarray into json recognisable int, float and lists
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

def drop_columns_if_exist(df, columns):
    df = df.copy()
    for col in columns:
        if col in df.columns:
            df = df.drop(columns=col)
    return df

def createDF(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
        
        category = findCategory(data)
        images = findImages(data)
        # nos_image = images['id'].max()
        df = findAnnotations(data)
        df = df.merge(images[['id','file_name']], left_on='image_id', right_on='id')
        df = df.rename(columns={'id_x': 'id'})
        df = drop_columns_if_exist(df,columns=['iscrowd','attributes','id_y'])
        return category, images, df

def fix_image_id(image1, image2, df, df2):
    
    image_comb = pd.concat([image1, image2], ignore_index = True)                              # combine image 1 and 2
    # sort and find unique image names
    image_comb = image_comb.sort_values(by=['file_name']).reset_index(drop=True)               # sort by image name
    image_new = image_comb.drop_duplicates(subset=['file_name'])                               # Get unique image names
    image_new = image_new.reset_index(drop=True)                                               # reset index
    image_new['id'] = image_new.index + 1                                                      # create new image id

    image1 = image1.merge(image_new[['id', 'file_name']], on='file_name', how='left')        # map image_id to image1 and image2
    image2 = image2.merge(image_new[['id', 'file_name']], on='file_name', how='left')        # map image_id to image1 and image2

    # use new id in image to replace old image_id in df
    df = df.merge(image1[['id_x', 'id_y']], left_on='image_id', right_on='id_x', how='left')
    df2 = df2.merge(image2[['id_x', 'id_y']], left_on='image_id', right_on='id_x', how='left')
    df = df.drop(columns=['image_id', 'id_x']).rename(columns={'id_y': 'image_id'})
    df2 = df2.drop(columns=['image_id', 'id_x']).rename(columns={'id_y': 'image_id'})

    # Make good the dfs
    df['iscrowd'] = 0
    df['attributes'] = [{'occluded':False}] * len(df['id'])
    df = df.drop(columns=['file_name'])
    df2['iscrowd'] = 0
    df2['attributes'] = [{'occluded':False}] * len(df2['id'])
    df2 = df2.drop(columns=['file_name'])

    return image_new, df, df2

def polygon_to_mask(polygons, height, width):
    '''
    Args:
        polygon (list): in [[...]]
        height, width (int): height and width of image
    Returns:
        np.array: a mask of shape (H, W)
    '''
    the_mask = np.zeros((height, width), dtype=np.uint8)

    # tackle the problem of having >1 polygon in an instance
    if len(polygons) > 1:
        for poly in polygons:
            # poly = np.array(poly).reshape((-1, 2))
            rles = cocomask.frPyObjects([poly], height, width)
            a_mask = np.squeeze(cocomask.decode(rles))
            # print(a_mask.shape)
            the_mask = np.logical_or(the_mask, a_mask)
    else:
        rles = cocomask.frPyObjects(polygons, height, width)
        the_mask = np.squeeze(cocomask.decode(rles))

    return the_mask

def bbox_convert(bbox):
    x1, y1, w, h = bbox
    bbox_new = [x1, y1, x1+w, y1+h]
    return bbox_new

# V1. Visualise detected instances on images

In [6]:
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import Boxes, Instances
import torch

gt_file = "../data/A12AL/A12AL_val4_SS4.json"
vis_dir = '../images/vis_gt/'
register_coco_instances("4vis", {}, gt_file, "../images/train_all")

category, images, df = createDF(gt_file)

dataset_vis = DatasetCatalog.get('4vis')
dataset_vis_metadata = MetadataCatalog.get('4vis')

for d in dataset_vis:
    d_name = os.path.basename(d["file_name"])
    img = cv2.imread(d["file_name"])
    height, width = img.shape[:2]
    visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_vis_metadata, scale=1)
    df_related = df[df['file_name'] == d_name]

    # draw_instance_predictions requires an Instances object in torch
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if len(df_related) != 0:
        temp_instances = Instances((height,width)).to(device)
        pred_boxes = [torch.tensor(bbox_convert(row['bbox']), dtype=torch.float32) for index, row in df_related.iterrows()] # present bbox as [x1, y1, x2, y2] and convert to tensor
        temp_instances.pred_boxes = Boxes(torch.stack(pred_boxes))
        if 'score' in df.columns:                           
            temp_instances.scores = torch.tensor([row['score'] for index, row in df_related.iterrows()])
        temp_instances.pred_classes = torch.tensor([row['category_id'] - 1 for index, row in df_related.iterrows()])        # metedata starts from 0
        pred_masks = [torch.tensor(polygon_to_mask(row['segmentation'], height, width)) for index, row in df_related.iterrows()]
        temp_instances.pred_masks = torch.stack(pred_masks)
        out = visualizer.draw_instance_predictions(temp_instances)
    else:                                                                       # if no instances, draw gt
        out = visualizer.draw_dataset_dict(d)

    os.makedirs(vis_dir, exist_ok=True)
    (filepath, filename) = os.path.split(d['file_name'])
    save_name = os.path.join(vis_dir,filename)
    cv2.imwrite(save_name, out.get_image()[:, :, ::-1])
    

In [7]:
# clear previous registry

dataset_to_clear = ['4vis']

for i in dataset_to_clear:
    DatasetCatalog.remove(i)
    MetadataCatalog.remove(i)